### 1. Load the dataset of sentiment140.csv into memory

In [54]:
import pandas as pd
import numpy as np
sentiment_data = pd.read_csv('/Users/mj/Desktop/DSCI 314 (Text Mining)/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', header = None)
# Reducing dataframe size
sentiment_data = sentiment_data.sample(frac=0.01)
sentiment_data.head(10)

,0,1,2,3,4,5
968114,4,1827836862,Sun May 17 11:32:58 PDT 2009,NO_QUERY,uk_cam,@Claire_Cordon gender is just one way that lan...
71229,0,1693896470,Mon May 04 00:12:28 PDT 2009,NO_QUERY,caaaaaity,"missed the train , that tends to happen to me ..."
646698,0,2236575372,Fri Jun 19 04:02:46 PDT 2009,NO_QUERY,littleemilyjane,off school sick mum refused to let me go to s...
1011093,4,1881066754,Fri May 22 03:29:17 PDT 2009,NO_QUERY,ClareOz,whaat a loaad a rubbish @mpisthename howsit? x
321808,0,2003928077,Tue Jun 02 07:23:04 PDT 2009,NO_QUERY,LolaOnTwatter,i wish i could eat a huge bowl of capn crunch ...
1201765,4,1985734317,Sun May 31 17:40:19 PDT 2009,NO_QUERY,wisdompathart,@Marge_Inovera awww. sending you some hugs and...
1345043,4,2043910645,Fri Jun 05 08:35:38 PDT 2009,NO_QUERY,vickyjones91,i hate buses. so need to pass my driving test
1459800,4,2063786886,Sun Jun 07 04:04:19 PDT 2009,NO_QUERY,zebslc,@SarahMillican75 you make yourself up to look ...
839791,4,1559517560,Sun Apr 19 11:20:57 PDT 2009,NO_QUERY,creena26,getting ready to go to the beach
1204046,4,1986191693,Sun May 31 18:28:04 PDT 2009,NO_QUERY,none01,"watching the MTV movie Awards, it's 3 am in Ge..."


### 2. Clean and preprocess the texts

In [55]:
# Selecting features
sentiment_data = sentiment_data.iloc[:, [0, 3, 5]]
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 968114 to 1144664
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       16000 non-null  int64 
 1   3       16000 non-null  object
 2   5       16000 non-null  object
dtypes: int64(1), object(2)
memory usage: 500.0+ KB


In [56]:
# Creating columns
sentiment_cols = [0, 3, 5]
# Adding new column 6 which is polarity forecast
sentiment_data[6] = sentiment_data[sentiment_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 968114 to 1144664
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       16000 non-null  int64 
 1   3       16000 non-null  object
 2   5       16000 non-null  object
 3   6       16000 non-null  object
dtypes: int64(1), object(3)
memory usage: 625.0+ KB


In [58]:
sentiment_data = sentiment_data.iloc[:, [0, 3]]
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 968114 to 1144664
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       16000 non-null  int64 
 1   6       16000 non-null  object
dtypes: int64(1), object(1)
memory usage: 375.0+ KB


In [59]:
percent_missing = sentiment_data.isnull().sum() * 100 / len(sentiment_data)
percent_missing

0    0.0
6    0.0
dtype: float64

### 3. Build the first model based on pipeline using the support vector machines

In [142]:
# Encoding labels into numerical values
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
 
sentiment_data[0] = le.fit_transform(sentiment_data.iloc[:, [0]].values.ravel())
sentiment_data[0].value_counts(normalize = True)

0
1    0.502375
0    0.497625
Name: proportion, dtype: float64

In [143]:
# Splitting dataset
from sklearn.model_selection import train_test_split

# Create features and label
X = sentiment_data[0].astype(str)
y = sentiment_data[6].astype(str)

# Giving dimensions of test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
print(f'X_train dimension: {X_train.shape}; y_train dimension: {y_train.shape}')
print(f'X_test dimension: {X_test.shape}; y_train dimension: {y_test.shape}')

X_train dimension: (12800,); y_train dimension: (12800,)
X_test dimension: (3200,); y_train dimension: (3200,)


In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import svm

# override the string tokenization step while preserving the preprocessing and n-grams generation steps
tfidf_vector = TfidfVectorizer(ngram_range = (1,1))

# support vector machine classifier
classifier = svm.SVC()

# create pipeline based on the cleaner, vectorizer, and classifier
pipeline = Pipeline([("cleaner", features()),
                    ("vectorizer", tfidf_vector),
                    ("classifier", classifier)])

# fit the model
pipeline.fit(X_train, y_train)

ValueError: empty vocabulary; perhaps the documents only contain stop words

### 4. Check the first model. Is it a good model based on the selected evaluation metrics? Please justify your answer

In [145]:
from sklearn.metrics import classification_report
# Specify target names since the y is label encodes using 0, 1 and 2
target_names = ['negative', 'neutral', 'positive']
# Predict the test data
y_pred = pipeline.predict(X_test)
# Print out the report
print(classification_report(y_test, y_pred, target_names = target_names))

NotFittedError: Pipeline is not fitted yet.